In [ ]:
from langchain_docling import DoclingLoader

In [ ]:
sourece = "ReAct.pdf"
loader = DoclingLoader(
    file_path=sourece,
    export_type = "markdown"
)
documents = loader.load()

In [ ]:
print(documents[0].page_content[:500])  # Print the first 500 characters of the first document

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Docling 마크다운 출력용 섹션 단위 분할
text_splitter = RecursiveCharacterTextSplitter(
    # 청크 최대 크기: 1200자
    # - 영어 논문 1200자 ≈ 300토큰 (대부분 임베딩 모델 512토큰 제한 내)
    # - 논문 섹션(Abstract, Introduction) 전체를 한 청크로 유지 가능
    # - 너무 작으면 문맥 손실, 너무 크면 검색 정확도 하락
    chunk_size=1200,
    
    # 청크 간 겹치는 부분: 200자 (chunk_size의 16%)
    # - 섹션 경계에서 문맥이 끊기는 것 방지
    # - 검색 시 경계 부근 정보 놓치지 않도록
    # - 일반적으로 chunk_size의 10-20% 권장
    chunk_overlap=200,
    
    # 분할 우선순위 (위에서부터 순서대로 시도)
    separators=[
        "\n## ",      # 1순위: 논문 섹션 헤더 (## Abstract, ## Introduction)
                      # Docling이 주요 섹션을 2레벨 헤더로 변환하므로
                      # 가장 큰 의미 단위인 섹션을 먼저 보존
        
        "\n### ",     # 2순위: 서브섹션 (### 3.1 Dataset, ### 3.2 Model)
                      # 섹션이 1200자 초과 시 서브섹션 단위로 자연스럽게 분할
        
        "\n\n",       # 3순위: 문단 경계
                      # 서브섹션도 길면 문단 단위로 분할
                      # 논문은 문단이 논리적 단위이므로 여기서 자르는 게 자연스러움
        
        "\n",         # 4순위: 줄바꿈
                      # 문단도 길면 줄 단위로
                      # 수식, 리스트, 코드 블록 등은 줄바꿈으로 구분됨
        
        ". ",         # 5순위: 문장 끝 (마침표 + 공백)
                      # 최소한 완전한 문장은 유지
                      # 공백까지 포함해야 문장 끝 정확히 감지
        
        " ",          # 6순위: 단어 경계 (공백)
                      # 문장도 초과하면 단어 단위로
                      # 최소한 단어는 쪼개지지 않게
        
        ""            # 7순위: 강제 분할 (문자 단위)
                      # 긴 URL, 코드, 수식 등 chunk_size 초과 시 문자로 강제 분할
                      # 에러 방지용 최후의 안전장치
    ],
    
    # 원본 문서에서의 시작 위치를 메타데이터에 추가
    # - RAG 검색 후 출처 추적 가능 (몇 번째 문자/페이지에서 왔는지)
    # - chunk.metadata['start_index']로 접근
    # - 사용자에게 "논문 3페이지 Introduction 섹션" 같은 정보 제공 가능
    add_start_index=True,
)
chunks = text_splitter.split_documents(documents)

In [ ]:
print(chunks[2])

In [ ]:
from FlagEmbedding import BGEM3FlagModel

In [ ]:
embedding_model = BGEM3FlagModel("BAAI/bge-m3")
embedding_model.model.to("cuda")
embedding_model.model.half()

In [ ]:
print(next(embedding_model.model.parameters()).device)
print(next(embedding_model.model.parameters()).dtype)

In [ ]:
texts = [chunk.page_content for chunk in chunks]

In [ ]:
print(texts)

In [ ]:
len(texts)

In [ ]:
outputs = embedding_model.encode(
    texts,
    batch_size = 12,
    max_length = 8192,
    return_dense = True,
    return_sparse = True,
    return_colbert_vecs=False,
)

In [ ]:
outputs['dense_vecs'][0][:5]

In [ ]:
list(outputs['lexical_weights'][0].items())[:5] 

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection
client = OpenSearch(
    hosts = [
        {
            "host": "https://search-papergraph-fonrzpiqlo6hbcwkufdwuukd6y.us-east-1.es.amazonaws.com",
            "port": 443
        }
        ],
    http_auth = ('wnghks5432', 'Wnghks1278@'),
    use_ssl = True,
    verify_certs = False
)

In [ ]:
index_name = "papers"

index_body = {
    "settings":{
        "index":{
            "knn": True,
            "knn.algo_param.ef_search":100
        }
    },
    "mappings":{
        "properties":{
            "text":{
                "type":"text",
                "analyzer":"standard"
            },
            "metadata":{
                "type":"object",
                "enabled": True
            },
            "dense_vector":{
                "type":"knn_vector",
                "dimension": 1024,
                "method":{
                    "name":"hnsw",
                    "space_type":"cosinesimil",
                    "engine":"faiss",
                    "parameters":{
                        "ef_construction":128,
                        "m":16
                    }
                }
            },
            "sparse_vector":{
                "type":"rank_features",
            }
        }
    }
}

In [ ]:
if client.indices.exists(index_name):
    client.indices.delete(index=index_name)
    
response = client.indices.create(index=index_name, body=index_body)

In [ ]:
from tqdm import tqdm

# bulk 색인용 액션 리스트 생성
# RAG 시스템을 처음 구축하거나 대량의 문서를 벡터화해서 넣을 때
actions = []
for i,chunk in tqdm(enumerate(chunks,desc="Indexing documents")):
    action = {
        "_index": index_name,
        "_id": i,
        "_body":{
            "text": chunk.page_content,
            "metadata": chunk.metadata,
            "dense_vector": outputs['dense_vecs'][i].tolist(),
            "sparse_vector": outputs['sparse_vecs'][i],
        }
    }
    actions.append(action)



In [ ]:
success,failed = client.bulk(
    client,
    actions,
    chunk_size=500,
    request_timeout=60
)
print(f"Successfully indexed {success} documents, failed to index {failed} documents.")

In [ ]:
# 개별 문서 색인
# 사용자가 게시글 하나를 작성해서 즉시 검색 엔진에 반영해야 할 때
for i,chunk in enumerate(chunks):
    doc = {
        "text": chunk.page_content,
        "metadata": chunk.metadata,
        "dense_vector": outputs['dense_vecs'][i].tolist(),
        "sparse_vector": outputs['sparse_vecs'][i],
    }
    response = client.index(
        index=index_name,
        id=i,
        body=doc,
        refresh=False
    )
    
client.indices.refresh(index=index_name)


In [ ]:
client.http.put(
    "/_search/pipeline/hybrid_norm_rrf",
    body={
        "response_processors": [
            {
                "normalization-processor": {
                    "normalization": {
                        "technique": "min_max"
                    },
                    "combination": {
                        "technique": "rrf",
                        "parameters": {
                            "rank_constant": 60
                        }
                    }
                }
            }
        ]
    }
)

In [ ]:
# 생성된 Pipeline 조회
pipeline = client.http.get("/_search/pipeline/hybrid_norm_rrf")
print(pipeline)

In [ ]:
def search_papers(query, top_k=5):
    """
    Hybrid Search 함수

    Args:
        query (str): 검색 쿼리 (예: "attention mechanism")
        top_k (int): 반환할 문서 개수 (기본 5개)

    Returns:
        list[dict]: 검색 결과 리스트
            - text: 청크 내용
            - metadata: 메타데이터
            - score: 검색 점수 (높을수록 관련성 높음)
    """

    # 1. 쿼리 인코딩 (BGE-M3)
    query_output = embedding_model.encode(
        [query],
        return_dense=True,
        return_sparse=True
    )

    # 2. Hybrid Search 쿼리 구성
    search_body = {
        "size": top_k,

        # 벡터는 반환하지 않음 (용량 절약)
        "_source": ["text", "metadata"],

        "query": {
            "hybrid": {
                "queries": [
                    # ===== Dense 검색 =====
                    {
                        "knn": {
                            "dense_vector": {
                                # 쿼리 벡터
                                "vector": query_output['dense_vecs'][0].tolist(),

                                # 찾을 이웃 개수
                                "k": top_k
                            }
                        }
                    },

                    # ===== Sparse 검색 =====
                    {
                        "bool": {
                            "should": [
                                {
                                    "rank_feature": {
                                        # sparse_vector.단어 형태로 필드명 구성
                                        "field": f"sparse_vector.{word}",

                                        # 쿼리에서의 해당 단어 가중치
                                        "boost": score
                                    }
                                }
                                # 쿼리의 모든 중요 단어에 대해 반복
                                for word, score in query_output['lexical_weights'][0].items()
                            ]
                        }
                    }
                ]
            }
        }
    }

    # 3. 검색 실행
    response = client.search(
        index=index_name,
        body=search_body,
        params = {"search_pipeline":"hybrid_norm_rrf"}
    )

    # 4. 결과 파싱
    results = []
    for hit in response["hits"]["hits"]:
        results.append({
            "text": hit["_source"]["text"],
            "metadata": hit["_source"]["metadata"],
            "score": hit["_score"]
        })

    return results


In [ ]:
# 검색 예시
query = "transformer attention mechanism"
results = search_papers(query, top_k=3)

print(f"\n🔍 검색어: '{query}'")
print(f"📊 결과: {len(results)}개\n")

for i, result in enumerate(results, 1):
    print(f"{'='*80}")
    print(f"📄 결과 {i} | 점수: {result['score']:.4f}")
    print(f"{'='*80}")

    # 텍스트 (처음 300자만)
    text = result['text']
    print(f"텍스트:\n{text[:300]}...")

    # 메타데이터
    print(f"\n메타데이터:")
    for key, value in result['metadata'].items():
        print(f"  - {key}: {value}")

    print()
